# Demo of using NMF for Mouse Brain ISH data

In [1]:
import numpy as np
import sklearn
from nmf_with_missing_values import nmf_with_missing_values
import ipyvolume

## load the data

In [2]:
tmp = np.load('../data/mouse_brain_ISH_float32.npz')
data = tmp['data']
sections = tmp['sections']
original_shape = data.shape
d = data.shape[1] * data.shape[2] * data.shape[3]
data = np.reshape(data, (data.shape[0], d))

## calculate nmf (could be quite long, do not run this chunk if you have the intermediate result

In [4]:
nmf = nmf_with_missing_values(n_outer_loops = 4, n_components = 18, init = 'nndsvd', random_state = None)
D = nmf.fit_transform(data)
A = nmf.components_

## save the intermediate result

In [5]:
np.savez('tmp_result.npz', D = D, A = A, X_guess = nmf.X_guess, data = data, original_shape = original_shape)

## load the intermediate result 

In [8]:
f = np.load('tmp_result.npz')
A = f['A']
original_shape = f['original_shape']
X_guess = f['X_guess']

## see the imputing effect

In [9]:
def make_cube(p1):
    ''' make X a cube
    '''
    max_dim = np.max(p1.shape)
    p2 = np.zeros((max_dim, max_dim, max_dim))
    for i in range(p1.shape[0]):
        for j in range(p1.shape[1]):
            for k in range(p1.shape[2]):
                p2[i,j,k] = max(p1[i,j,k],0)
    return p2

In [10]:
p1 = np.reshape(X_guess[0,:] - np.maximum(data[0,:],0), original_shape[1:])
p2 = make_cube(p1)
ipyvolume.figure()
ipyvolume.volshow(p2, lighting=True)
ipyvolume.show()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


## visualize the pattern

In [11]:
p1 = np.reshape(A[0,:], original_shape[1:])
p2 = make_cube(p1)
ipyvolume.figure()
ipyvolume.volshow(p2, lighting=True)
ipyvolume.show()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


## store the trained model using pickle

In [24]:
import pickle
nmf.X_guess = None
with open('nmf.pickle', 'wb') as f:
    pickle.dump(nmf, f)

In [26]:
import sklearn.decomposition
help(sklearn.decomposition.NMF)

Help on class NMF in module sklearn.decomposition.nmf:

class NMF(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin)
 |  Non-Negative Matrix Factorization (NMF)
 |  
 |  Find two non-negative matrices (W, H) whose product approximates the non-
 |  negative matrix X. This factorization can be used for example for
 |  dimensionality reduction, source separation or topic extraction.
 |  
 |  The objective function is::
 |  
 |      0.5 * ||X - WH||_Fro^2
 |      + alpha * l1_ratio * ||vec(W)||_1
 |      + alpha * l1_ratio * ||vec(H)||_1
 |      + 0.5 * alpha * (1 - l1_ratio) * ||W||_Fro^2
 |      + 0.5 * alpha * (1 - l1_ratio) * ||H||_Fro^2
 |  
 |  Where::
 |  
 |      ||A||_Fro^2 = \sum_{i,j} A_{ij}^2 (Frobenius norm)
 |      ||vec(A)||_1 = \sum_{i,j} abs(A_{ij}) (Elementwise L1 norm)
 |  
 |  For multiplicative-update ('mu') solver, the Frobenius norm
 |  (0.5 * ||X - WH||_Fro^2) can be changed into another beta-divergence loss,
 |  by changing the beta_loss parameter.
 |  
 | 